Assignment 14

In [11]:
import numpy as np
import cv2
import math



# Get the risk area
def GetRiskAreaImage(dataset):
    data = []
    with open(dataset, 'r') as f:
        data = f.readlines()
    # strip the new line character
    data = [x.strip() for x in data]

    # for each character in the data get int value
    for i in range(len(data)):
        data[i] = [int(x) for x in data[i]]

    return np.array(data, dtype=np.uint8)

def GetRisk(map, node):
    return map[node.x][node.y]

class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __repr__(self):
        return 'Node({}, {})'.format(self.x, self.y)
class Route:
    def __init__(self, nodes, risk):
        self.nodes = [x for x in nodes]
        self.risk = risk
    def AddNode(self, node):
        self.nodes.append(node)
    def Contains(self, node):
        return node in self.nodes

def GetAdjacentNodes(map, current):
    adjacentNodes = []
    adjacentNodes.append(Node(current.x, current.y - 1))
    adjacentNodes.append(Node(current.x, current.y + 1))
    adjacentNodes.append(Node(current.x - 1, current.y))
    adjacentNodes.append(Node(current.x + 1, current.y))

    # Remove Nodes that are not in the map
    adjacentNodes = [x for x in adjacentNodes if x.x >= 0 and x.x < map.shape[0] and x.y >= 0 and x.y < map.shape[1]]
    return adjacentNodes

def showScaledImage(image, breaktime=0, scaling = 1, name='image'):
    Image = np.zeros((image.shape[0]*scaling,image.shape[1]*scaling), dtype=np.uint8)
    for y in range(image.shape[0]): 
        for x in range(image.shape[1]):
            if image[y][x] == 1:
                Image[y*scaling:(y+1)*scaling, x*scaling:(x+1)*scaling] = 255
    #cv2.imwrite(f'./images/aoc11_[{i}].bmp', Image)
    cv2.imshow(name,Image)
    cv2.waitKey(breaktime)

# Recursive function that will return the path of lowest risk. 
def FindPathWithLowestRisk( map, current, end,  currentRoute, pathTree, exploredNodesMap, optimalPath = None):
    if current.x < 0 or current.x >= map.shape[0] or current.y < 0 or current.y >= map.shape[1]:
        return None
    if current == end:
        return currentRoute
    else:
        

        # Get the adjacent nodes
        adjacentNodes = GetAdjacentNodes(map, current)

        # remove nodes that are already explored
        adjacentNodes = [x for x in adjacentNodes if exploredNodesMap[x.x][x.y] == 0]
        
        # sort the adjacent nodes by distance to the end
        adjacentNodes = sorted(adjacentNodes, key=lambda x: GetRisk(map, x))

        #print(adjacentNodes)

        for node in adjacentNodes:
            newCurrentRoute = Route(currentRoute.nodes, currentRoute.risk)
            newCurrentRoute.AddNode(node)
            newCurrentRoute.risk += GetRisk(map, node)
            #pathTree.append(newCurrentRoute)
            newExploredNodesMap = np.copy(exploredNodesMap)
            newExploredNodesMap[node.x][node.y] = 1
            if None != optimalPath and newCurrentRoute.risk > optimalPath.risk:
                continue
            showScaledImage(newExploredNodesMap, breaktime=1, scaling = 5, name='exploredNodesMap')
            

            foundRoute = FindPathWithLowestRisk(map, newCurrentRoute.nodes[-1], end, newCurrentRoute, pathTree, newExploredNodesMap, optimalPath)
            
            if foundRoute != None:
                if optimalPath == None or (optimalPath.risk > foundRoute.risk and foundRoute.nodes[-1] == end):
                    optimalPath = foundRoute
                    showScaledImage(MarkRoute(map.shape, optimalPath), breaktime=1, scaling = 5, name='optimalPath')
        return optimalPath    


def FindPathWithLowestRisk2( map, current, end,  currentRoute, pathTree,exploredNodesMap, optimalPath = None):
    if current.x < 0 or current.x >= map.shape[0] or current.y < 0 or current.y >= map.shape[1]:
        return None
    if current == end:
        return currentRoute
    if exploredNodesMap[current.x][current.y] == 1:
        return None

    else:
        
        #print(current)
        # Get the adjacent nodes
        adjacentNodes = GetAdjacentNodes(map, current)

        # remove nodes that are already explored
        adjacentNodes = [x for x in adjacentNodes if exploredNodesMap[x.x,x.y] == False]
        #print(adjacentNodes)
        # sort the adjacent nodes by distance to the end
        adjacentNodes = sorted(adjacentNodes, key=lambda x: (math.sqrt((x.x - end.x)**2 + (x.y - end.y)**2)+1)*GetRisk(map, x))
        
        #print(adjacentNodes)
        for node in adjacentNodes:
            newCurrentRoute = Route(currentRoute.nodes, currentRoute.risk)
            newCurrentRoute.risk += GetRisk(map, node)
            newCurrentRoute.AddNode(node)
            exploredNodesMap[current.x,current.y] = 1
            if None != optimalPath and newCurrentRoute.risk > optimalPath.risk:
                continue
            else: pathTree.append(newCurrentRoute)
        
        # sorth pathTree by risk and distance
        while len(pathTree) > 0:
            #print("GrabNext")
            pathTree = sorted(pathTree, key=lambda x: (math.sqrt((x.nodes[-1].x - end.x)**2 + (x.nodes[-1].y - end.y)**2)+1)*(GetRisk(map,x.nodes[-1]) )+ x.risk)
            #print([x.risk for x in pathTree])
            currentRoute = pathTree.pop(0)
            #print([x.risk for x in pathTree])
            
            showScaledImage(exploredNodesMap, breaktime=1, scaling = 5, name='exploredNodesMap')
            foundRoute = FindPathWithLowestRisk2(map, currentRoute.nodes[-1], end, currentRoute, pathTree, exploredNodesMap, optimalPath)
            
            if foundRoute != None:
                if optimalPath == None or (optimalPath.risk > foundRoute.risk and foundRoute.nodes[-1] == end):
                    optimalPath = foundRoute
                    showScaledImage(MarkRoute(map.shape, optimalPath), breaktime=1, scaling = 5, name='optimalPath')
            
        return optimalPath      
        
                
            

def MarkRoute(shape,route):
    map = np.zeros(shape, dtype=np.uint8)
    for node in route.nodes:
        map[node.x][node.y] = 1
    return map





In [12]:
if(True):
    map = GetRiskAreaImage('./data/aoc15_test1.txt')

    startRoute = Route([Node(0,0)], 0)
    explored = np.zeros(map.shape)
    route = FindPathWithLowestRisk2(map, Node(0,0), Node(map.shape[0]-1,map.shape[1]-1), startRoute , [startRoute], explored)

    MarkedRoute = MarkRoute(map.shape, route)

    showScaledImage(MarkedRoute, breaktime=0, scaling = 20, name= "Result")

    print(f'Risk: {route.risk}')

KeyboardInterrupt: 

In [ ]:
if(False):
    map = GetRiskAreaImage('./data/aoc15.txt')

    startRoute = Route([Node(0,0)], 0)
    explored = np.zeros(map.shape)
    route = FindPathWithLowestRisk2(map, Node(0,0), Node(map.shape[0]-1,map.shape[1]-1), startRoute , [startRoute], explored)

    MarkedRoute = MarkRoute(map.shape, route)

    print(f'Risk: {route.risk}')
    showScaledImage(explored, breaktime=0, scaling = 20)
    showScaledImage(MarkedRoute, breaktime=0, scaling = 20)